In [7]:
# Same code is used in Houdini to test, with some modifications in imports and without some hard-coded values
# So here I'm separating imports and definitions of those hardcoded values.

import pickle
import os
import numpy as np
import math

data_dir = os.path.normpath("samplesGeneration/data")
file_names = os.listdir(data_dir)
file_name = file_names[51]
file_path = os.path.join(data_dir, file_name)

print(data_dir, "\n", file_path)

samplesGeneration/data 
 samplesGeneration/data/10.740195295589409_140.0


In [9]:
import itertools
from tensorflow.python.data import AUTOTUNE

with open(file_path, "rb") as f:
    data = pickle.load(f)

def angle_to_sin_cos(angle):
    return [math.sin(angle), math.cos(angle)]

def sin_cos_to_angle(sin, cos):
    return math.atan2(sin, cos)

def angle_to_sin_and_back(angle):
    return math.atan2(*angle_to_sin_cos(angle))

def to_abs_with_sin_cos(arr):
    result = list()
    for box in arr:
        angles = [angle_to_sin_cos(angle) for angle in box[3:]]
        result.append(list(box[:3]) + list(itertools.chain(*angles)))
    return np.asarray(result)

def process_abs_sample(sample):
    """ Changes sample in form of single 6 elements array to houdini compatible vec3 pos and vec4 orient """
    orient = [sin_cos_to_angle(sample[3+2*x], sample[4+2*x]) for x in range(3)] + [1]
    return sample[:3], orient

def configure_for_performance(ds):
    ds = ds.cache()
    ds = ds.shuffle(buffer_size=1000)
    ds = ds.batch(32)
    ds = ds.prefetch(buffer_size=AUTOTUNE)
    return ds

class DataProcessor(object):
    """ Code responsible for processing of data gathered into a class for readability.

    Trying few different approaches to data representation to test what works best:

    For every frame n following representations are calculated:
    - x_delta: array (4, 6) of delta between frames n and n-1: (Δx, Δy, Δz, Δα , Δβ, Δγ)
    - x_abs_n_minus_1: array (2, 4, 9) of absolute values in frame n-1
    - x_abs_n: x_abs in in frame n
      Structure of x_abs_*: [[x, y, z, sin(α), cos(α), sin(β), cos(β), sin(γ), cos(γ)], [...]]
    - x_force: array of forces in frame n: (power, sin(Φ), cos(Φ)

    - y_delta: array (4, 6) of delta between frames n and n+1
    - y_abs: array (4, 9) of absolute values in frame n+1

    Data is passed to neural networks in forms:
    - x: (x_delta, x_force) y: y_delta
    - x: (x_delta, x_abs_n, x_force) y: y_delta
    - x: (x_delta, x_abs_n, x_force) y: y_abs
    - x: (x_abs_n_minus_1, x_abs_n, x_force) y: y_abs
    - x: (x_abs_n_minus_1, x_abs_n, x_force) y: y_delta
    """
    def __init__(self, paths):
        # self.x_delta = np.empty((1, 4, 6))
        # self.x_abs_n_minus_1 = np.empty((1, 4, 9))
        # self.x_abs_n = np.empty((1, 4, 9))
        # self.x_force = np.empty((1, 3))
        #
        # self.y_delta = np.empty((1, 4, 6))
        # self.y_abs = np.empty((1, 4, 9))

        self.x_delta = list()
        self.x_abs_n_minus_1 = list()
        self.x_abs_n = list()
        self.x_force = list()

        self.y_delta = list()
        self.y_abs = list()

        for file_path in paths:
            with open(file_path, "rb") as f:
                frames = pickle.load(f)
                filename = os.path.basename(file_path)
                force, force_angle = [float(part) for part in filename.split("_")]
                last_frame = None
                for num, (frame, next_frame) in enumerate(zip(frames[:-1], frames[1:])):
                    # print(num, end="\r")
                    if num <=2:
                        frame_force = 0
                    else:
                        frame_force = force
                    if last_frame is not None:
                        x_force = np.asarray(angle_to_sin_cos(force_angle) + [frame_force])
                        n_minus_1_abs = np.asarray([f[0] + f[1][:-1] for f in last_frame])
                        n_abs = np.asarray([f[0] + f[1][:-1] for f in frame])
                        n_plus_1_abs = np.asarray([f[0] + f[1][:-1] for f in next_frame])

                        x_delta = n_abs - n_minus_1_abs
                        y_delta = n_plus_1_abs - n_abs

                        n_minus_1_abs = to_abs_with_sin_cos(n_minus_1_abs)
                        n_abs = to_abs_with_sin_cos(n_abs)
                        y_abs = to_abs_with_sin_cos(n_plus_1_abs)

                        self.x_delta.append(x_delta)
                        self.x_abs_n_minus_1.append(n_minus_1_abs)
                        self.x_abs_n.append(n_abs)
                        self.x_force.append(x_force)

                        self.y_delta.append(y_delta)
                        self.y_abs.append(y_abs)
                    last_frame = frame

        self.x_delta = np.asarray(self.x_delta)
        self.x_abs_n_minus_1 = np.asarray(self.x_abs_n_minus_1)
        self.x_abs_n = np.asarray(self.x_abs_n)
        self.x_force = np.asarray(self.x_force)

        self.y_delta = np.asarray(self.y_delta)
        self.y_abs = np.asarray(self.y_abs)

dp = DataProcessor([file_path])
# process_sample(data, file_name)

In [10]:
dp.x_delta.shape

(238, 4, 6)